# **0. Load libraries**

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
#!pip install implicit
from implicit.als import AlternatingLeastSquares
from sklearn.metrics import precision_score, ndcg_score
import warnings
warnings.filterwarnings('ignore')

In [13]:
# og_file_path="~/code/Alanoudis/food-delivery-rec/data/updated_data/orders_sg25k.txt"
# og_data = pd.read_csv(og_file_path, sep=',', encoding='utf-8')
# og_data.head()

# og_products = "~/code/Alanoudis/food-delivery-rec/data/raw_data/products_sg.txt"
# og_products = pd.read_csv(og_products, sep=',', encoding='utf-8')
# og_products.head()

# og_vendors = "~/code/Alanoudis/food-delivery-rec/data/raw_data/vendors_sg.txt"
# og_vendors = pd.read_csv(og_vendors, sep=',', encoding='utf-8')
# og_vendors.head()

In [2]:
# old 100k data, im looking for products_rating and order_frequency columns
full_data100k_path = "~/code/Alanoudis/food-delivery-rec/data/test-train/full_data100k.csv"
full_data100k = pd.read_csv(full_data100k_path, index_col=0)
full_data100k.head()

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_day,name,unit_price,chain_id,vendor_geohash,cuisine_origin,order_frequency,product_rating
0,1ba124d4e5,w21z7,0,212753d2,783e85338f1c,0,12:03:29,85 days,japanese garlic karaage don,6.0,66c9978d,w21z7,japanese,1,4
1,1ba124d4e5,w21z7,0,212753d2,084ab73246e6,0,12:03:29,85 days,chicken cutlet don,6.8,66c9978d,w21z7,japanese,1,5
2,1ba124d4e5,w21z7,0,212753d2,30eba3cc2676,0,12:03:29,85 days,beef sukiyaki don,6.8,66c9978d,w21z7,japanese,1,3
3,1ba124d4e5,w21z7,0,212753d2,3910309eea60,0,12:03:29,85 days,japanese beef yakiniku don,6.8,66c9978d,w21z7,japanese,1,5
4,1ba124d4e5,w21z7,0,212753d2,20049fb602cb,0,12:03:29,85 days,teriyaki salmon don,8.0,66c9978d,w21z7,japanese,1,5


In [3]:
full_data100k.shape

(100000, 15)

In [4]:
#full_data includes each product in each order + new columns for time features
full_data_path = "~/code/Alanoudis/food-delivery-rec/notebooks/alanoud/full_data_complete.csv"
full_data = pd.read_csv(full_data_path)
full_data.head()

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_day,name,unit_price,...,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,order_hour,meal_of_day,order_hour_sin,order_hour_cos
0,1ba124d4e5,w21z7,0,212753d2,783e85338f1c,0,1900-01-01 12:03:29,85,japanese garlic karaage don,6.0,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0
1,1ba124d4e5,w21z7,0,212753d2,084ab73246e6,0,1900-01-01 12:03:29,85,chicken cutlet don,6.8,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0
2,1ba124d4e5,w21z7,0,212753d2,30eba3cc2676,0,1900-01-01 12:03:29,85,beef sukiyaki don,6.8,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0
3,1ba124d4e5,w21z7,0,212753d2,3910309eea60,0,1900-01-01 12:03:29,85,japanese beef yakiniku don,6.8,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0
4,1ba124d4e5,w21z7,0,212753d2,20049fb602cb,0,1900-01-01 12:03:29,85,teriyaki salmon don,8.0,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0


In [5]:
full_data.columns

Index(['customer_id', 'customer_geohash', 'order_id', 'vendor_id',
       'product_id', 'day_of_week', 'order_time', 'order_day', 'name',
       'unit_price', 'chain_id', 'vendor_geohash', 'cuisine_origin',
       'is_weekend', 'dow_0', 'dow_1', 'dow_2', 'dow_3', 'dow_4', 'dow_5',
       'dow_6', 'order_hour', 'meal_of_day', 'order_hour_sin',
       'order_hour_cos'],
      dtype='object')

In [6]:
full_data.shape

(100000, 25)

In [7]:
#full_data2 includes 1 row per order. This is used for vendor ratings.
vendor_ratings_path = "~/code/Alanoudis/food-delivery-rec/notebooks/alanoud/Alshaimas-rating/vendor_ratings.csv"
full_data2 = pd.read_csv(vendor_ratings_path)
full_data2 = full_data2.rename(columns={'avg_vendor_rating': 'vendor_rating'})
full_data2.head()

,customer_id,customer_geohash,order_id,vendor_id,day_of_week,order_time,order_day,chain_id,vendor_geohash,cuisine_origin,vendor_rating,num_products,total_order_value,products_ordered
0,00119c8178,w21zu,39095,e7cb5902,2,16:30:07,10 days,ef3142e8,w21zu,malaysian,3.5,1,5.6,Seafood Fried Rice
1,00198e01e4,w21z6,35939,02acaff6,0,13:57:35,85 days,8c51e46b,w21z6,chinese,3.9,4,8.0,"Bean Curd Skin 凉拌腐竹, Dried Beancurd 凉拌素鸡, Blac..."
2,001a5689fc,w21z3,48288,a2c60e71,0,20:10:35,78 days,dd69fe77,w21z3,singaporean,4.5,1,2.4,Carrot Cake (Melur)
3,001a5689fc,w21z3,48289,a3bc472c,4,20:50:40,40 days,c59edb7d,w21z9,snacks,4.0,1,1.6,Served without Fries
4,001a5689fc,w21z3,48290,de18b671,3,20:05:42,53 days,f88ffd2b,w21z3,american,4.5,1,4.0,WHOPPER® Meal


In [8]:
df = full_data.copy()
df.head()

,customer_id,customer_geohash,order_id,vendor_id,product_id,day_of_week,order_time,order_day,name,unit_price,...,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,order_hour,meal_of_day,order_hour_sin,order_hour_cos
0,1ba124d4e5,w21z7,0,212753d2,783e85338f1c,0,1900-01-01 12:03:29,85,japanese garlic karaage don,6.0,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0
1,1ba124d4e5,w21z7,0,212753d2,084ab73246e6,0,1900-01-01 12:03:29,85,chicken cutlet don,6.8,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0
2,1ba124d4e5,w21z7,0,212753d2,30eba3cc2676,0,1900-01-01 12:03:29,85,beef sukiyaki don,6.8,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0
3,1ba124d4e5,w21z7,0,212753d2,3910309eea60,0,1900-01-01 12:03:29,85,japanese beef yakiniku don,6.8,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0
4,1ba124d4e5,w21z7,0,212753d2,20049fb602cb,0,1900-01-01 12:03:29,85,teriyaki salmon don,8.0,...,False,False,False,False,False,False,12,lunch,1.224647e-16,-1.0


In [9]:
# add products rating to full_data
# One-line solution:
# Insert 'order_frequency' before 'product_rating'
full_data.insert(full_data.columns.get_loc('chain_id'), 'order_frequency', full_data100k['order_frequency'])

# Then insert 'product_rating' right after it
full_data.insert(full_data.columns.get_loc('chain_id'), 'product_rating', full_data100k['product_rating'])

print("✅ Done! order_frequency and product_rating added between unit_price and chain_id")
print(f"full_data now has {full_data.shape[1]} columns")

✅ Done! order_frequency and product_rating added between unit_price and chain_id
full_data now has 27 columns


## Inspect both dataframes

In [10]:
# Quick data exploration
print("Data shape:", full_data.shape)
print("\nUnique counts:")
print(f"Customers: {full_data['customer_id'].nunique()}")
print(f"Vendors: {full_data['vendor_id'].nunique()}")
print(f"Cuisines: {full_data['cuisine_origin'].nunique()}")
print(f"Orders: {full_data['order_id'].nunique()}")

print("\nRating distribution:")
print(full_data['product_rating'].value_counts().sort_index())

# Check vendor_ratings dataframe
print(f"\nVendor ratings shape: {full_data2.shape}")
print(f"Vendor ratings columns: {full_data2.columns.tolist()}")
print(f"Vendor ratings stats:")
print(full_data2['vendor_rating'].describe())

Data shape: (100000, 27)

Unique counts:
Customers: 11174
Vendors: 5818
Cuisines: 18
Orders: 49029

Rating distribution:
product_rating
1.0    10733
2.0    13319
3.0    17395
4.0    18578
5.0    39974
Name: count, dtype: int64

Vendor ratings shape: (49029, 14)
Vendor ratings columns: ['customer_id', 'customer_geohash', 'order_id', 'vendor_id', 'day_of_week', 'order_time', 'order_day', 'chain_id', 'vendor_geohash', 'cuisine_origin', 'vendor_rating', 'num_products', 'total_order_value', 'products_ordered']
Vendor ratings stats:
count    49029.000000
mean         4.073067
std          0.368664
min          2.500000
25%          3.900000
50%          4.000000
75%          4.400000
max          5.000000
Name: vendor_rating, dtype: float64


# 1. Implicit Feedback Matrix Features:
Weighted scores combining order frequency and ratings (Use orders as strong preference signals).

User-Vendor matrix: Interaction strength between customers and vendors (Customer’s favorite restaurants based on order frequency)

User-Cuisine matrix: Preference strength for cuisine types (“Who likes which types of food?")

In [11]:
#df.dropna(inplace=True)
df.isnull().sum()

customer_id          0
customer_geohash     0
order_id             0
vendor_id            0
product_id           0
day_of_week          0
order_time           0
order_day            0
name                32
unit_price           0
chain_id             0
vendor_geohash       0
cuisine_origin       0
is_weekend           0
dow_0                0
dow_1                0
dow_2                0
dow_3                0
dow_4                0
dow_5                0
dow_6                0
order_hour           0
meal_of_day          0
order_hour_sin       0
order_hour_cos       0
dtype: int64

In [12]:
df.dropna(inplace=True)
df.isnull().sum()

customer_id         0
customer_geohash    0
order_id            0
vendor_id           0
product_id          0
day_of_week         0
order_time          0
order_day           0
name                0
unit_price          0
chain_id            0
vendor_geohash      0
cuisine_origin      0
is_weekend          0
dow_0               0
dow_1               0
dow_2               0
dow_3               0
dow_4               0
dow_5               0
dow_6               0
order_hour          0
meal_of_day         0
order_hour_sin      0
order_hour_cos      0
dtype: int64

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
from implicit.als import AlternatingLeastSquares
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# =============================================
# DATA PREPROCESSING & FEATURE ENGINEERING
# =============================================

print("Step 1: Data Preprocessing...")

# Merge vendor ratings with main data
print("Merging vendor ratings...")
full_data_enhanced = full_data.merge(
    full_data2[['order_id', 'vendor_id', 'vendor_rating', 'total_order_value', 'num_products']],
    on=['order_id', 'vendor_id'],
    how='left',
    suffixes=('', '_vendor')
)

print(f"Enhanced data shape: {full_data_enhanced.shape}")

# Filter by product rating >= 4 (as requested)
print("Filtering by rating >= 4...")
# Change from 4 to 3 to include more data
data_filtered = full_data_enhanced[full_data_enhanced['product_rating'] >= 3].copy()
print(f"After rating filter: {data_filtered.shape}")

# Create order-level aggregated dataset
print("Creating order-level features...")
order_level_data = data_filtered.groupby('order_id').agg({
    'customer_id': 'first',
    'vendor_id': 'first',
    'cuisine_origin': 'first',
    'day_of_week': 'first',
    'order_day': 'first',
    'order_hour': 'first',
    'meal_of_day': 'first',
    'is_weekend': 'first',
    'unit_price': 'sum',
    'product_id': 'count',
    'product_rating': 'mean',
    'vendor_rating': 'first',
    'order_frequency': 'first',
    'total_order_value': 'first',
    'num_products': 'first',
    'customer_geohash': 'first',
    'vendor_geohash': 'first'
}).rename(columns={
    'product_id': 'num_items',
    'product_rating': 'avg_product_rating',
    'unit_price': 'total_order_value_calculated'
}).reset_index()

# Use the provided total_order_value if available, otherwise use calculated
order_level_data['final_order_value'] = order_level_data['total_order_value'].fillna(
    order_level_data['total_order_value_calculated']
)

print(f"Order-level data shape: {order_level_data.shape}")

# Create vendor features
print("Creating vendor features...")
vendor_features = data_filtered.groupby('vendor_id').agg({
    'cuisine_origin': 'first',
    'vendor_geohash': 'first',
    'vendor_rating': 'mean',
    'product_rating': ['mean', 'count'],
    'order_id': 'nunique'
}).reset_index()

vendor_features.columns = [
    'vendor_id', 'cuisine_origin', 'vendor_geohash',
    'avg_vendor_rating', 'avg_product_rating', 'total_products', 'total_orders'
]

print(f"Vendor features shape: {vendor_features.shape}")

# =============================================
# INTERACTION MATRIX CONSTRUCTION
# =============================================

print("\nStep 2: Building Interaction Matrices...")

def create_interaction_matrices(df):
    """Create multiple interaction matrices for different aspects"""

    # 1. Primary User-Vendor Matrix (Frequency + Rating Weighted)
    user_vendor_interactions = df.groupby(['customer_id', 'vendor_id']).agg({
        'num_items': 'sum',
        'avg_product_rating': 'mean',
        'vendor_rating': 'mean',
        'order_id': 'count'
    }).reset_index()

    # Enhanced weighted score combining multiple factors
    user_vendor_interactions['weighted_score'] = (
        user_vendor_interactions['order_id'] *  # Frequency
        (user_vendor_interactions['avg_product_rating'] / 5.0) *  # Product quality
        (user_vendor_interactions['vendor_rating'] / 5.0)  # Vendor quality
    )

    user_vendor_matrix = user_vendor_interactions.pivot_table(
        index='customer_id',
        columns='vendor_id',
        values='weighted_score',
        fill_value=0
    )

    # 2. User-Cuisine Preference Matrix
    user_cuisine_interactions = df.groupby(['customer_id', 'cuisine_origin']).agg({
        'order_id': 'count',
        'avg_product_rating': 'mean',
        'vendor_rating': 'mean'
    }).reset_index()

    user_cuisine_interactions['preference_score'] = (
        user_cuisine_interactions['order_id'] *
        (user_cuisine_interactions['avg_product_rating'] / 5.0) *
        (user_cuisine_interactions['vendor_rating'] / 5.0)
    )

    user_cuisine_matrix = user_cuisine_interactions.pivot_table(
        index='customer_id',
        columns='cuisine_origin',
        values='preference_score',
        fill_value=0
    )

    # 3. Temporal Preference Matrix (Time-based patterns)
    temporal_interactions = df.groupby(['customer_id', 'meal_of_day']).agg({
        'order_id': 'count',
        'final_order_value': 'mean'
    }).reset_index()

    temporal_matrix = temporal_interactions.pivot_table(
        index='customer_id',
        columns='meal_of_day',
        values='order_id',
        fill_value=0
    )

    return {
        'user_vendor': user_vendor_matrix,
        'user_cuisine': user_cuisine_matrix,
        'temporal': temporal_matrix,
        'raw_interactions': user_vendor_interactions
    }

matrices = create_interaction_matrices(order_level_data)
user_vendor_matrix = matrices['user_vendor']
user_cuisine_matrix = matrices['user_cuisine']

print(f"User-Vendor Matrix: {user_vendor_matrix.shape}")
print(f"User-Cuisine Matrix: {user_cuisine_matrix.shape}")

# =============================================
# COLLABORATIVE FILTERING WITH ALS
# =============================================

print("\nStep 3: Training Collaborative Filtering Model...")

def train_als(interaction_matrix, factors=50, iterations=15):
    """Train ALS model for collaborative filtering"""

    sparse_matrix = csr_matrix(interaction_matrix.values)

    als_model = AlternatingLeastSquares(
        factors=factors,
        iterations=iterations,
        regularization=0.01,
        random_state=42
    )

    als_model.fit(sparse_matrix)

    return als_model, sparse_matrix

# Train initial model on full data for reference
als_model_full, sparse_matrix_full = train_als(user_vendor_matrix)

print(f"ALS Model trained: {als_model_full.user_factors.shape} user factors, {als_model_full.item_factors.shape} item factors")

# =============================================
# CONTENT-BASED USER PROFILES
# =============================================

print("\nStep 4: Building User Profiles...")

def create_comprehensive_user_profiles(df):
    """Create comprehensive user profiles with multiple preference dimensions"""

    # 1. Cuisine Preference Profiles
    cuisine_prefs = df.groupby(['customer_id', 'cuisine_origin']).agg({
        'order_id': 'count',
        'avg_product_rating': 'mean',
        'vendor_rating': 'mean',
        'final_order_value': 'mean'
    }).reset_index()

    cuisine_prefs['preference_score'] = (
        cuisine_prefs['order_id'] *
        (cuisine_prefs['avg_product_rating'] / 5.0) *
        (cuisine_prefs['vendor_rating'] / 5.0)
    )

    # Normalize by user
    user_totals = cuisine_prefs.groupby('customer_id')['preference_score'].transform('sum')
    cuisine_prefs['normalized_score'] = cuisine_prefs['preference_score'] / user_totals

    user_cuisine_profiles = cuisine_prefs.pivot_table(
        index='customer_id',
        columns='cuisine_origin',
        values='normalized_score',
        fill_value=0
    )

    # 2. Temporal Behavior Profiles
    temporal_profiles = df.groupby('customer_id').agg({
        'order_hour': ['mean', 'std'],
        'is_weekend': 'mean',
        'day_of_week': lambda x: x.mode()[0] if len(x.mode()) > 0 else 3,
        'meal_of_day': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'lunch'
    }).reset_index()

    temporal_profiles.columns = ['customer_id', 'avg_order_hour', 'std_order_hour',
                               'weekend_ratio', 'fav_day', 'fav_meal']

    # 3. Spending & Frequency Profiles
    spending_profiles = df.groupby('customer_id').agg({
        'final_order_value': ['mean', 'std', 'max'],
        'num_items': 'mean',
        'order_id': 'count',
        'order_frequency': 'mean',
        'vendor_rating': 'mean'
    }).reset_index()

    spending_profiles.columns = ['customer_id', 'avg_order_value', 'std_order_value',
                               'max_order_value', 'avg_items', 'total_orders',
                               'avg_order_frequency', 'avg_vendor_rating_choice']

    # 4. Combine all profiles
    user_profiles = user_cuisine_profiles.reset_index()

    # Merge temporal profiles
    user_profiles = user_profiles.merge(temporal_profiles, on='customer_id', how='left')

    # Merge spending profiles
    user_profiles = user_profiles.merge(spending_profiles, on='customer_id', how='left')

    return user_profiles.fillna(0)

user_profiles_full = create_comprehensive_user_profiles(order_level_data)
print(f"User profiles shape: {user_profiles_full.shape}")

# =============================================
# PERFORMANCE METRICS
# =============================================

class RecommendationMetrics:
    """Comprehensive recommendation evaluation metrics"""

    @staticmethod
    def precision_at_k(recommendations, actual_items, k=10):
        if len(recommendations) == 0:
            return 0.0
        recommended_set = set(recommendations[:k])
        actual_set = set(actual_items)
        return len(recommended_set & actual_set) / k

    @staticmethod
    def recall_at_k(recommendations, actual_items, k=10):
        if len(actual_items) == 0:
            return 0.0
        recommended_set = set(recommendations[:k])
        actual_set = set(actual_items)
        return len(recommended_set & actual_set) / len(actual_set)

    @staticmethod
    def ndcg_at_k(recommendations, actual_items, k=10):
        if len(recommendations) == 0 or len(actual_items) == 0:
            return 0.0

        relevance = [1 if item in actual_items else 0 for item in recommendations[:k]]

        dcg = sum([rel / np.log2(i + 2) for i, rel in enumerate(relevance)])

        ideal_relevance = [1] * min(len(actual_items), k)
        idcg = sum([rel / np.log2(i + 2) for i, rel in enumerate(ideal_relevance)])

        return dcg / idcg if idcg > 0 else 0.0

    @staticmethod
    def mrr(recommendations, actual_items):
        for rank, item in enumerate(recommendations, 1):
            if item in actual_items:
                return 1.0 / rank
        return 0.0

    @staticmethod
    def map_at_k(recommendations, actual_items, k=10):
        if len(actual_items) == 0:
            return 0.0

        precision_sum = 0.0
        num_relevant = 0

        for i, item in enumerate(recommendations[:k]):
            if item in actual_items:
                num_relevant += 1
                precision_sum += num_relevant / (i + 1)

        return precision_sum / min(len(actual_items), k)

    @staticmethod
    def coverage(recommended_items, all_items):
        return len(set(recommended_items)) / len(all_items) if len(all_items) > 0 else 0.0

# =============================================
# HYBRID RECOMMENDATION ENGINE
# =============================================

class HybridFoodRecommender:
    """Hybrid recommendation engine combining CF and content-based filtering"""

    def __init__(self, als_model, user_profiles, vendor_features, interaction_matrices):
        self.als_model = als_model
        self.user_profiles = user_profiles
        self.vendor_features = vendor_features.set_index('vendor_id')
        self.matrices = interaction_matrices
        self.user_vendor_matrix = interaction_matrices['user_vendor']
         # Precompute fallback scores for efficiency
        self.fallback_scores = self._compute_fallback_scores()

    def _compute_fallback_scores(self):
        """Precompute fallback scores combining popularity and ratings"""
        popularity = self.user_vendor_matrix.sum(axis=0)  # Overall popularity
        vendor_ratings = self.vendor_features['avg_vendor_rating']

        # Normalize both metrics
        popularity_norm = popularity / popularity.max()
        rating_norm = vendor_ratings / 5.0

        # Combined score: 60% popularity + 40% rating
        fallback_df = pd.DataFrame({
            'vendor_id': popularity.index,
            'fallback_score': 0.6 * popularity_norm + 0.4 * rating_norm
        })

        return fallback_df.sort_values('fallback_score', ascending=False)

    def generate_recommendations_with_fallback(self, user_id, top_n=10,
                                            cf_weight=0.6, cb_weight=0.25, pop_weight=0.15,
                                            diversity_penalty=0.1, fallback_threshold=0.01):
        """
        Enhanced version with robust fallback mechanism
        """
        try:
            # Try personalized recommendations first
            personalized_recs = self.generate_recommendations(
                user_id, top_n, cf_weight, cb_weight, pop_weight, diversity_penalty
            )

            # Check if personalized recommendations are good enough
            if len(personalized_recs) >= top_n and personalized_recs['final_score'].max() > fallback_threshold:
                return personalized_recs
            else:
                # Fall back to popular + highly-rated vendors
                print(f"Using fallback for user {user_id} (low personalized scores)")
                return self._get_fallback_recommendations(top_n)

        except Exception as e:
            # If any error occurs, use fallback
            print(f"Error in personalized recommendations for {user_id}: {e}. Using fallback.")
            return self._get_fallback_recommendations(top_n)

    def _get_fallback_recommendations(self, top_n=10):
        """Get fallback recommendations based on popularity and ratings"""
        fallback_recs = self.fallback_scores.head(top_n * 2).copy()

        # Add vendor details
        fallback_recs['vendor_rating'] = fallback_recs['vendor_id'].map(
            self.vendor_features['avg_vendor_rating']
        )
        fallback_recs['cuisine'] = fallback_recs['vendor_id'].map(
            self.vendor_features['cuisine_origin']
        )

        # Add dummy scores for consistency
        fallback_recs['final_score'] = fallback_recs['fallback_score']
        fallback_recs['cf_score'] = 0
        fallback_recs['cb_score'] = 0
        fallback_recs['diversity_factor'] = 1.0

        return fallback_recs.head(top_n)
    # def generate_recommendations(self, user_id, top_n=10,
    #                            cf_weight=0.6, cb_weight=0.25, pop_weight=0.15,
    #                            diversity_penalty=0.1):
    #     """
    #     Generate hybrid recommendations with multiple scoring components
    #     """
    #     # 1. Collaborative Filtering Scores
    #     if user_id in self.user_vendor_matrix.index:
    #         user_idx = self.user_vendor_matrix.index.get_loc(user_id)
    #         cf_scores = self.als_model.item_factors.dot(self.als_model.user_factors[user_idx])
    #     else:
    #         # New user - use popularity
    #         cf_scores = self.user_vendor_matrix.sum(axis=0).values

    #     # Create candidate dataframe
    #     candidates = pd.DataFrame({
    #         'vendor_id': self.user_vendor_matrix.columns,
    #         'cf_score': cf_scores
    #     })

    #     # Get top candidates for efficiency
    #     candidates = candidates.nlargest(top_n * 5, 'cf_score')

    #     # 2. Content-Based Filtering Scores
    #     cb_scores = {}
    #     vendor_cuisines = {}

    #     if user_id in self.user_profiles['customer_id'].values:
    #         user_profile = self.user_profiles[self.user_profiles['customer_id'] == user_id].iloc[0]

    #         for vendor_id in candidates['vendor_id']:
    #             if vendor_id in self.vendor_features.index:
    #                 vendor_cuisine = self.vendor_features.loc[vendor_id, 'cuisine_origin']
    #                 vendor_cuisines[vendor_id] = vendor_cuisine

    #                 if vendor_cuisine in user_profile:
    #                     cb_scores[vendor_id] = user_profile[vendor_cuisine]
    #                 else:
    #                     cb_scores[vendor_id] = 0
    #             else:
    #                 cb_scores[vendor_id] = 0
    #                 vendor_cuisines[vendor_id] = 'unknown'
    #     else:
    #         # New user - neutral content scores
    #         for vendor_id in candidates['vendor_id']:
    #             cb_scores[vendor_id] = 0.3
    #             if vendor_id in self.vendor_features.index:
    #                 vendor_cuisines[vendor_id] = self.vendor_features.loc[vendor_id, 'cuisine_origin']
    #             else:
    #                 vendor_cuisines[vendor_id] = 'unknown'

    #     candidates['cb_score'] = candidates['vendor_id'].map(cb_scores)
    #     candidates['cuisine'] = candidates['vendor_id'].map(vendor_cuisines)

    #     # 3. Popularity Scores (for new users and diversity)
    #     popularity_scores = self.user_vendor_matrix.sum(axis=0)
    #     candidates['pop_score'] = candidates['vendor_id'].map(
    #         popularity_scores / popularity_scores.max()
    #     ).fillna(0)

    #     # 4. Vendor Quality Scores
    #     vendor_ratings = self.vendor_features['avg_vendor_rating']
    #     candidates['vendor_rating'] = candidates['vendor_id'].map(vendor_ratings).fillna(0)
    #     candidates['quality_score'] = candidates['vendor_rating'] / 5.0

    #     # 5. Normalize all scores
    #     for score_col in ['cf_score', 'cb_score', 'pop_score', 'quality_score']:
    #         max_val = candidates[score_col].max()
    #         if max_val > 0:
    #             candidates[f'{score_col}_norm'] = candidates[score_col] / max_val
    #         else:
    #             candidates[f'{score_col}_norm'] = 0

    #     # 6. Combine scores with weights
    #     candidates['hybrid_score'] = (
    #         cf_weight * candidates['cf_score_norm'] +
    #         cb_weight * candidates['cb_score_norm'] +
    #         pop_weight * candidates['pop_score_norm'] +
    #         0.1 * candidates['quality_score_norm']  # Small weight for quality
    #     )

    #     # 7. Apply diversity: penalize same cuisine recommendations
    #     final_recommendations = []
    #     cuisine_counts = {}

    #     for _, candidate in candidates.nlargest(top_n * 3, 'hybrid_score').iterrows():
    #         cuisine = candidate['cuisine']
    #         cuisine_count = cuisine_counts.get(cuisine, 0)

    #         # Apply diversity penalty
    #         diversity_factor = 1.0 / (1.0 + diversity_penalty * cuisine_count)
    #         final_score = candidate['hybrid_score'] * diversity_factor

    #         final_recommendations.append({
    #             'vendor_id': candidate['vendor_id'],
    #             'final_score': final_score,
    #             'cf_score': candidate['cf_score'],
    #             'cb_score': candidate['cb_score'],
    #             'vendor_rating': candidate['vendor_rating'],
    #             'cuisine': cuisine,
    #             'diversity_factor': diversity_factor
    #         })

    #         cuisine_counts[cuisine] = cuisine_count + 1

    #     final_df = pd.DataFrame(final_recommendations)
    #     return final_df.nlargest(top_n, 'final_score')

# =============================================
# MODEL EVALUATION FRAMEWORK
# =============================================

print("\nStep 5: Setting up Evaluation Framework...")

# Temporal train-test split
def temporal_train_test_split(df, time_col='order_day', test_size=0.2):
    """Split data temporally"""
    df_sorted = df.sort_values(time_col)
    split_idx = int(len(df_sorted) * (1 - test_size))

    train_df = df_sorted.iloc[:split_idx]
    test_df = df_sorted.iloc[split_idx:]

    return train_df, test_df

train_orders, test_orders = temporal_train_test_split(order_level_data)
print(f"Train orders: {len(train_orders)}, Test orders: {len(test_orders)}")

# Train on training data only for evaluation
print("Training on training split for evaluation...")
train_matrices = create_interaction_matrices(train_orders)
als_model_train, _ = train_als(train_matrices['user_vendor'])
train_user_profiles = create_comprehensive_user_profiles(train_orders)

# Initialize recommender for evaluation
evaluation_recommender = HybridFoodRecommender(
    als_model_train,
    train_user_profiles,
    vendor_features,
    train_matrices
)

def evaluate_recommender(recommender, test_df, sample_users=100, top_k=10):
    """Comprehensive evaluation of the recommender - FIXED VERSION"""
    print(f"\nEvaluating on {sample_users} test users...")

    metrics = RecommendationMetrics()
    test_users = test_df['customer_id'].unique()

    if sample_users and sample_users < len(test_users):
        test_users = np.random.choice(test_users, sample_users, replace=False)

    all_results = []
    all_recommendations = []

    for i, user_id in enumerate(test_users):
        if i % 20 == 0:
            print(f"Progress: {i}/{len(test_users)}")

        user_test_data = test_df[test_df['customer_id'] == user_id]
        actual_vendors = user_test_data['vendor_id'].unique().tolist()

        if len(actual_vendors) == 0:
            continue

        try:
            recommendations = recommender.generate_recommendations_with_fallback(user_id, top_n=top_k*2)
            rec_vendors = recommendations['vendor_id'].tolist()

            results = {
                'user_id': user_id,
                'num_actual': len(actual_vendors),
                'precision@5': metrics.precision_at_k(rec_vendors, actual_vendors, 5),
                'precision@10': metrics.precision_at_k(rec_vendors, actual_vendors, 10),
                'recall@5': metrics.recall_at_k(rec_vendors, actual_vendors, 5),
                'recall@10': metrics.recall_at_k(rec_vendors, actual_vendors, 10),
                'ndcg@5': metrics.ndcg_at_k(rec_vendors, actual_vendors, 5),
                'ndcg@10': metrics.ndcg_at_k(rec_vendors, actual_vendors, 10),
                'mrr': metrics.mrr(rec_vendors, actual_vendors),
                'map@10': metrics.map_at_k(rec_vendors, actual_vendors, 10)
            }

            all_results.append(results)
            all_recommendations.extend(rec_vendors[:top_k])

        except Exception as e:
            print(f"Error processing user {user_id}: {e}")
            continue

    if all_results:
        results_df = pd.DataFrame(all_results)

        # FIX: Only compute mean for numeric columns, exclude 'user_id'
        numeric_columns = [col for col in results_df.columns if col != 'user_id']
        avg_metrics = results_df[numeric_columns].mean()

        # Calculate coverage
        all_vendors = train_matrices['user_vendor'].columns.tolist()
        coverage = metrics.coverage(all_recommendations, all_vendors)

        print("\n" + "="*70)
        print("COMPREHENSIVE EVALUATION RESULTS")
        print("="*70)

        metric_groups = {
            'Precision': ['precision@5', 'precision@10'],
            'Recall': ['recall@5', 'recall@10'],
            'Ranking Quality': ['ndcg@5', 'ndcg@10', 'mrr', 'map@10']
        }

        for group, metrics_list in metric_groups.items():
            print(f"\n{group}:")
            for metric in metrics_list:
                if metric in avg_metrics:
                    print(f"  {metric:15}: {avg_metrics[metric]:.4f}")

        print(f"\nCoverage: {coverage:.4f}")
        print(f"Average actual interactions per user: {results_df['num_actual'].mean():.2f}")
        print(f"Number of users evaluated: {len(results_df)}")

        return results_df, avg_metrics, coverage
    else:
        print("No results to evaluate")
        return pd.DataFrame(), {}, 0.0

# Run evaluation
results_df, avg_metrics, coverage = evaluate_recommender(evaluation_recommender, test_orders, sample_users=100)

# =============================================
# FINAL PRODUCTION MODEL
# =============================================

print("\nTraining final production model on full data...")
final_matrices = create_interaction_matrices(order_level_data)
final_als_model, _ = train_als(final_matrices['user_vendor'])
final_user_profiles = create_comprehensive_user_profiles(order_level_data)

# Create final production recommender
production_recommender = HybridFoodRecommender(
    final_als_model,
    final_user_profiles,
    vendor_features,
    final_matrices
)

print("✅ Production recommendation system ready!")

# =============================================
# SAMPLE RECOMMENDATIONS & ANALYSIS
# =============================================

print("\n" + "="*70)
print("SAMPLE RECOMMENDATIONS FROM PRODUCTION MODEL")
print("="*70)

# Test with active users
active_users = order_level_data['customer_id'].value_counts().head(3).index.tolist()

for user_id in active_users:
    print(f"\nRecommendations for user {user_id}:")
    try:
        recommendations = production_recommender.generate_recommendations(user_id, top_n=5)

        # Check user's order history
        user_history = order_level_data[order_level_data['customer_id'] == user_id]
        user_vendors = user_history['vendor_id'].tolist()
        user_cuisines = user_history['cuisine_origin'].unique().tolist()

        print(f"User history: {len(user_history)} orders, {len(user_vendors)} unique vendors, cuisines: {user_cuisines}")

        for i, (_, rec) in enumerate(recommendations.iterrows(), 1):
            is_ordered = "✓" if rec['vendor_id'] in user_vendors else " "

            print(f"  {i}. {rec['vendor_id']} | Score: {rec['final_score']:.4f} | "
                  f"Cuisine: {rec['cuisine']} | Rating: {rec['vendor_rating']:.2f} | "
                  f"Ordered Before: [{is_ordered}]")

    except Exception as e:
        print(f"  Error generating recommendations for {user_id}: {e}")

# =============================================
# MODEL ANALYSIS AND INSIGHTS
# =============================================

print("\n" + "="*70)
print("MODEL INSIGHTS AND ANALYSIS")
print("="*70)

# Analyze the model performance
print(f"\nModel Statistics:")
print(f"- Total users: {len(order_level_data['customer_id'].unique())}")
print(f"- Total vendors: {len(vendor_features)}")
print(f"- Total orders: {len(order_level_data)}")
print(f"- Cuisines covered: {len(order_level_data['cuisine_origin'].unique())}")

# Check data sparsity
total_possible_interactions = len(final_matrices['user_vendor']) * len(final_matrices['user_vendor'].columns)
actual_interactions = (final_matrices['user_vendor'] > 0).sum().sum()
sparsity = 1 - (actual_interactions / total_possible_interactions)

print(f"\nData Sparsity Analysis:")
print(f"Total possible user-vendor interactions: {total_possible_interactions:,}")
print(f"Actual interactions: {actual_interactions:,}")
print(f"Sparsity: {sparsity:.4f} ({sparsity*100:.2f}%)")

# Vendor popularity distribution
vendor_popularity = final_matrices['user_vendor'].sum(axis=0)
print(f"\nVendor Popularity Distribution:")
print(f"Most popular vendor: {vendor_popularity.max():.2f} interactions")
print(f"Least popular vendor: {vendor_popularity.min():.2f} interactions")
print(f"Average vendor popularity: {vendor_popularity.mean():.2f} interactions")

# Performance summary
if len(results_df) > 0:
    print("\n" + "="*70)
    print("PERFORMANCE SUMMARY")
    print("="*70)

    print(f"Overall Precision@10: {avg_metrics.get('precision@10', 0):.4f}")
    print(f"Overall Recall@10: {avg_metrics.get('recall@10', 0):.4f}")
    print(f"Overall NDCG@10: {avg_metrics.get('ndcg@10', 0):.4f}")
    print(f"Coverage: {coverage:.4f}")

    # Interpretation
    print(f"\nInterpretation:")
    precision_10 = avg_metrics.get('precision@10', 0)
    if precision_10 > 0.3:
        print("✅ Excellent performance! More than 30% of recommendations are relevant.")
    elif precision_10 > 0.2:
        print("✅ Good performance! About 20-30% of recommendations are relevant.")
    elif precision_10 > 0.1:
        print("✅ Reasonable performance for a food delivery context.")
    else:
        print("🔍 Performance can be improved. Consider feature engineering or model tuning.")

# Save the production model for future use
print(f"\nProduction model is ready to use!")
print(f"Use 'production_recommender.generate_recommendations(user_id)' to get recommendations.")

Step 1: Data Preprocessing...
Merging vendor ratings...
Enhanced data shape: (100000, 30)
Filtering by rating >= 4...
After rating filter: (75947, 30)
Creating order-level features...
Order-level data shape: (42251, 19)
Creating vendor features...
Vendor features shape: (5656, 7)

Step 2: Building Interaction Matrices...
User-Vendor Matrix: (10176, 5656)
User-Cuisine Matrix: (10176, 18)

Step 3: Training Collaborative Filtering Model...


  0%|          | 0/15 [00:00<?, ?it/s]

ALS Model trained: (10176, 50) user factors, (5656, 50) item factors

Step 4: Building User Profiles...
User profiles shape: (10176, 31)

Step 5: Setting up Evaluation Framework...
Train orders: 33800, Test orders: 8451
Training on training split for evaluation...


  0%|          | 0/15 [00:00<?, ?it/s]


Evaluating on 100 test users...
Progress: 0/100
Progress: 20/100
Progress: 40/100
Progress: 60/100
Progress: 80/100

COMPREHENSIVE EVALUATION RESULTS

Precision:
  precision@5    : 0.0380
  precision@10   : 0.0240

Recall:
  recall@5       : 0.0870
  recall@10      : 0.1178

Ranking Quality:
  ndcg@5         : 0.0737
  ndcg@10        : 0.0834
  mrr            : 0.1197
  map@10         : 0.0554

Coverage: 0.0798
Average actual interactions per user: 1.86
Number of users evaluated: 100

Training final production model on full data...


  0%|          | 0/15 [00:00<?, ?it/s]

✅ Production recommendation system ready!

SAMPLE RECOMMENDATIONS FROM PRODUCTION MODEL

Recommendations for user 50acf8536f:
User history: 348 orders, 348 unique vendors, cuisines: ['american', 'chinese', 'singaporean', 'asian', 'snacks', 'japanese', 'thai', 'italian']
  1. 4a7d6a4f | Score: 0.9808 | Cuisine: chinese | Rating: 4.25 | Ordered Before: [✓]
  2. a8ee414f | Score: 0.8428 | Cuisine: singaporean | Rating: 4.22 | Ordered Before: [✓]
  3. 91704ac5 | Score: 0.7899 | Cuisine: japanese | Rating: 4.33 | Ordered Before: [✓]
  4. 60257fe4 | Score: 0.7504 | Cuisine: singaporean | Rating: 4.01 | Ordered Before: [✓]
  5. 61081731 | Score: 0.7223 | Cuisine: chinese | Rating: 3.98 | Ordered Before: [✓]

Recommendations for user f40e63eabc:
User history: 133 orders, 133 unique vendors, cuisines: ['snacks']
  1. e09f6517 | Score: 1.0972 | Cuisine: snacks | Rating: 4.53 | Ordered Before: [✓]
  2. 3c30529e | Score: 0.5805 | Cuisine: snacks | Rating: 4.06 | Ordered Before: [✓]
  3. f12a5bb5 |

### Different Selection Strategy

In [ ]:
# =============================================
# BETTER USER SELECTION STRATEGIES
# =============================================

def select_test_users(df, strategy="diverse", n_users=5):
    """
    Select users for testing recommendations based on different strategies
    """

    # Calculate user order counts
    user_order_counts = df.groupby('customer_id').size()

    if strategy == "diverse":
        # Select users with different order frequencies
        return {
            'new_user': user_order_counts[user_order_counts == 1].index[0],      # 1 order
            'medium_user': user_order_counts[user_order_counts.between(2, 5)].index[0],  # 2-5 orders
            'power_user': user_order_counts[user_order_counts > 10].index[0],    # 10+ orders
            'random_user': df['customer_id'].sample(1).iloc[0],                  # Random
            'first_user': df['customer_id'].iloc[0]                              # Your current method
        }

    elif strategy == "power_users":
        # Users with most orders (best for testing CF)
        return user_order_counts.nlargest(n_users).index.tolist()

    elif strategy == "new_users":
        # Users with few orders (tests CB fallback)
        return user_order_counts.nsmallest(n_users).index.tolist()

    elif strategy == "coverage":
        # Users who ordered different cuisines
        user_cuisine_diversity = df.groupby('customer_id')['cuisine_origin'].nunique()
        return user_cuisine_diversity.nlargest(n_users).index.tolist()

# Test with diverse users
test_users = select_test_users(df, strategy="diverse")

print("🧪 TESTING WITH DIVERSE USERS:")
for user_type, user_id in test_users.items():
    user_orders = len(df[df['customer_id'] == user_id])
    user_cuisines = df[df['customer_id'] == user_id]['cuisine_origin'].nunique()

    print(f"{user_type}: {user_id} ({user_orders} orders, {user_cuisines} cuisines)")

    # Generate recommendations for this user
    recommendations = generate_recommendations(user_id, als_model, user_profiles, vendor_features)
    top_cuisine = vendor_features.loc[recommendations.iloc[0]['vendor_id'], 'cuisine_origin']

    print(f"  Top recommendation: {top_cuisine}")
    print(f"  CF score: {recommendations.iloc[0]['cf_score']:.6f}")
    print(f"  CB score: {recommendations.iloc[0]['cb_score']:.3f}")
    print()

🧪 TESTING WITH DIVERSE USERS:
new_user: 008ab40ac0 (1 orders, 1 cuisines)
  Top recommendation: chinese
  CF score: 0.000055
  CB score: 1.000

medium_user: 008ce71183 (3 orders, 2 cuisines)
  Top recommendation: american
  CF score: 0.045964
  CB score: 0.679

power_user: 046958a898 (12 orders, 4 cuisines)
  Top recommendation: snacks
  CF score: 0.039190
  CB score: 0.487

random_user: 046958a898 (12 orders, 4 cuisines)
  Top recommendation: snacks
  CF score: 0.039190
  CB score: 0.487

first_user: 008ab40ac0 (1 orders, 1 cuisines)
  Top recommendation: chinese
  CF score: 0.000055
  CB score: 1.000



In [13]:
# Let's investigate the power user more closely
power_user_id = '046958a898'
power_user_orders = df[df['customer_id'] == power_user_id]

print(f"🔍 ANALYZING POWER USER: {power_user_id}")
print("=" * 50)

# Check their actual order history
print("📋 ORDER HISTORY:")
cuisine_history = power_user_orders['cuisine_origin'].value_counts()
for cuisine, count in cuisine_history.items():
    print(f"  - {cuisine}: {count} orders")

# Check their preference profile
if power_user_id in user_profiles['customer_id'].values:
    user_profile = user_profiles[user_profiles['customer_id'] == power_user_id].iloc[0]
    print(f"\n🎯 PREFERENCE PROFILE:")
    # Get top 3 cuisine preferences
    cuisine_cols = [col for col in user_profile.index if col in df['cuisine_origin'].unique()]
    cuisine_prefs = user_profile[cuisine_cols].astype(float)
    top_preferences = cuisine_prefs.nlargest(3)
    for cuisine, score in top_preferences.items():
        print(f"  - {cuisine}: {score:.3f}")

# Check why snacks was recommended
snacks_recommendation = recommendations[recommendations['vendor_id'] == recommendations.iloc[0]['vendor_id']]
print(f"\n🤔 WHY SNACKS RECOMMENDATION?")
print(f"Vendor: {snacks_recommendation['vendor_id'].iloc[0]}")
print(f"CF Score: {snacks_recommendation['cf_score'].iloc[0]:.6f}")
print(f"CB Score: {snacks_recommendation['cb_score'].iloc[0]:.3f}")

🔍 ANALYZING POWER USER: 046958a898
📋 ORDER HISTORY:
  - snacks: 6 orders
  - american: 3 orders
  - italian: 2 orders
  - singaporean: 1 orders

🎯 PREFERENCE PROFILE:
  - snacks: 0.487
  - american: 0.249
  - italian: 0.188

🤔 WHY SNACKS RECOMMENDATION?
Vendor: 90cca11f
CF Score: 0.000055
CB Score: 1.000


In [ ]:
# =============================================
# COMPREHENSIVE MODEL EVALUATION
# =============================================

def evaluate_model_performance(df, als_model, user_profiles, vendor_features, n_test_users=10):
    """
    Evaluate recommendation quality across different user types
    """

    user_order_counts = df.groupby('customer_id').size()

    print("📊 MODEL PERFORMANCE EVALUATION")
    print("=" * 50)

    # Test different user segments
    segments = {
        'New Users (1 order)': user_order_counts[user_order_counts == 1].index[:3],
        'Medium Users (2-5 orders)': user_order_counts[user_order_counts.between(2, 5)].index[:3],
        'Power Users (6+ orders)': user_order_counts[user_order_counts >= 6].index[:3],
        'Diverse Cuisine Users': df.groupby('customer_id')['cuisine_origin'].nunique().nlargest(3).index
    }

    for segment_name, user_ids in segments.items():
        print(f"\n🎯 {segment_name}:")
        print("-" * 30)

        for user_id in user_ids:
            if user_id in user_vendor_matrix.index:
                recommendations = generate_recommendations(user_id, als_model, user_profiles, vendor_features)

                # Basic user stats
                user_orders = len(df[df['customer_id'] == user_id])
                user_cuisines = df[df['customer_id'] == user_id]['cuisine_origin'].nunique()

                # Recommendation stats
                top_rec = recommendations.iloc[0]
                top_cuisine = vendor_features.loc[top_rec['vendor_id'], 'cuisine_origin']

                print(f"  User {user_id[:8]}...: {user_orders} orders, {user_cuisines} cuisines")
                print(f"    → Top rec: {top_cuisine} (CF: {top_rec['cf_score']:.4f}, CB: {top_rec['cb_score']:.2f})")

# Run comprehensive evaluation
evaluate_model_performance(df, als_model, user_profiles, vendor_features)

📊 MODEL PERFORMANCE EVALUATION

🎯 New Users (1 order):
------------------------------
  User 008ab40a...: 1 orders, 1 cuisines
    → Top rec: chinese (CF: 0.0001, CB: 1.00)
  User 00ba08ba...: 1 orders, 1 cuisines
    → Top rec: american (CF: 0.0342, CB: 1.00)
  User 00e9c13b...: 1 orders, 1 cuisines
    → Top rec: american (CF: 0.0098, CB: 1.00)

🎯 Medium Users (2-5 orders):
------------------------------
  User 008ce711...: 3 orders, 2 cuisines
    → Top rec: american (CF: 0.0460, CB: 0.68)
  User 00c41737...: 2 orders, 2 cuisines
    → Top rec: chinese (CF: 0.0267, CB: 0.49)
  User 01058d1d...: 2 orders, 1 cuisines
    → Top rec: snacks (CF: 0.1304, CB: 1.00)

🎯 Power Users (6+ orders):
------------------------------
  User 01907d62...: 9 orders, 2 cuisines
    → Top rec: snacks (CF: 0.0977, CB: 0.65)
  User 027862ef...: 6 orders, 3 cuisines
    → Top rec: chinese (CF: 0.0216, CB: 0.66)
  User 028a29c2...: 6 orders, 2 cuisines
    → Top rec: snacks (CF: 0.0577, CB: 0.48)

🎯 Diverse 